In [11]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.offsetbox as offsetbox

import math
import random
from scipy.special import softmax

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import os
from PIL import Image
from collections import defaultdict

In [12]:
def eigen_cam(model, img, layer_name = None, label_index=None):
    """ Ce code permet est une implémentation de la méthode eigen_cam permettant de récupérer des cartes de saillance. 
        elles sont basées sur la décomposition de valeur de singulière. En gros on projette la sortie de la toute dernière couche de convolution 
        sur le premier vecteur singulier droit de l'équation. C'est la méthode avec laquelle j'ai récupéré les cartes qui sont dans ce dossier. 
        j'ai dû choisir cette méthode moins par choix que par praticité, en effet c'est une des rares à ne pas trop imposer de contraintes sur la couche 
        de sortie du réseau et donc la tâche effectuée. Puisque la couche de sortie de notre tâche d'identification est assez particulière et qu'elle n'effectue 
        pas une classification, c'était en fait la première méthode disponible. Donc, si on vous dit pendant votre soutenance que cette méthode 
        n'est pas adaptée (la SVD supposant la linéarité qui est une hypothèse baffouée en réseaux de neurones), vous pouvez répondre ça.  """
    # pour toute entrée, lui rajoute une dimension batch first si pas déjà présente:
    img = tf.convert_to_tensor(img) if not tf.is_tensor(img) else img
    img = tf.expand_dims(img, axis=0) if len(img.shape) == 3 else img # on travaille donc ici sur du (1, 224, 224, 3), càd du (batch, hauteur, largeur, canal)
    img_height, img_width = img.shape[1], img.shape[2]
    # extract the label_index of the last conv layer : 
    if layer_name is None:
        for layer in reversed(model.layers):
            if isinstance(layer, tf.keras.layers.Conv2D):
                layer_name = layer.name
                break
        if layer_name is None:
            raise ValueError("Aucune couche de convolution trouvée dans le modèle.")

    layer = model.get_layer(layer_name).output
    activation_model = tf.keras.Model(inputs = model.input, outputs = [model.output, layer])
    
    # 1°) Forward pass pour obtenir preds et activations de la layer
    preds, feature = activation_model(img)

    # 2°) décomposition en valeur singulière
    s, u, v = tf.linalg.svd(feature, full_matrices = True)
    vT = tf.transpose(v, [0, 1, 3, 2])

    # 3°) Calcul de la carte CAM : 
    # On multiplie d'abord s par vT : s est scalaire pour chaque composante singulière, cela revient à une multiplication element-wise
    scaled_vT = s[..., 0, None, None] * vT[..., 0, None, :]
    # Ensuite, on fait une unique multiplication matricielle avec u
    eigen_cam = tf.linalg.matmul(u[..., 0, None], scaled_vT)

    # 4°) On somme sur l'axe canal 
    eigen_cam = tf.reduce_sum(eigen_cam, axis = -1, keepdims = True)

    # 5°) Resize à l'espace (i,j) de l'image et normalisation
    eigen_cam = tf.image.resize(eigen_cam, (img_height, img_width))
    eigen_cam_min, eigen_cam_max = tf.reduce_min(eigen_cam), tf.reduce_max(eigen_cam)
    eigen_cam = (eigen_cam - eigen_cam_min) / (eigen_cam_max - eigen_cam_min)
   
    eigen_cam = tf.squeeze(eigen_cam, axis = 0)
    return eigen_cam

In [13]:
def create_autoencoder(input_latent, shape):
    input_img = tf.keras.Input(shape=shape)
    # Feature extractor (encoder)
    x = tf.keras.layers.BatchNormalization()(input_img)
    x = tf.keras.layers.Conv2D(8, kernel_size=11, padding='same', activation='relu', use_bias=False)(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(16, kernel_size=5, activation='relu',padding='same', use_bias=False)(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu',padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu',padding='same', use_bias=False)(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu',padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu',padding='same', use_bias=False)(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    # Espace latent
    encoded = x
    latent_space_layer = tf.keras.layers.Dense(units=1*1*input_latent)(encoded)
    latent_space_layer_norm = tf.keras.layers.BatchNormalization(name='latent_space_norm')(latent_space_layer)
    
    # Feature redear (decoder)
    x_recon = tf.keras.layers.Reshape(target_shape=(1,1,input_latent))(latent_space_layer_norm)
    x_recon = tf.keras.layers.Conv2DTranspose(64, kernel_size=5, activation='relu', strides=(2, 2), padding='same', use_bias=False)(x_recon)
    x_recon = tf.keras.layers.BatchNormalization()(x_recon)
    x_recon = tf.keras.layers.Conv2DTranspose(64, kernel_size=5, activation='relu', padding='same', use_bias=False)(x_recon)
    x_recon = tf.keras.layers.BatchNormalization()(x_recon)
    x_recon = tf.keras.layers.Conv2DTranspose(32, kernel_size=5, activation='relu', strides=(2, 2), padding='same', use_bias=False)(x_recon)
    x_recon = tf.keras.layers.BatchNormalization()(x_recon)
    x_recon = tf.keras.layers.Conv2DTranspose(32, kernel_size=5, activation='relu', padding='same', use_bias=False)(x_recon)
    x_recon = tf.keras.layers.BatchNormalization()(x_recon)
    x_recon = tf.keras.layers.Conv2DTranspose(16, kernel_size=5, activation='relu', strides=(2, 2), padding='same', use_bias=False)(x_recon)
    x_recon = tf.keras.layers.BatchNormalization()(x_recon)
    x_recon = tf.keras.layers.Conv2DTranspose(8, kernel_size=5, activation='relu', strides=(2, 2), padding='same', use_bias=False)(x_recon)
    x_recon = tf.keras.layers.BatchNormalization()(x_recon)
    x_recon = tf.keras.layers.Conv2DTranspose(1, kernel_size=11, activation='sigmoid', padding='same')(x_recon)  
    x_recon = tf.keras.layers.Resizing(28,28)(x_recon)

    # Classifier branch -> pour Théo uniquement, si assez de temps disponible
    # classifier = tf.keras.layers.Dense(10, activation='softmax')(latent_space_layer_norm)

    # Full model
    model = tf.keras.Model(inputs=input_img, outputs=[x_recon])
    # model = tf.keras.Model(inputs=input_img, outputs=[x_recon, classifier]) -> pour Théo uniquement, si assez de temps disponible
    model.compile(optimizer='SGD', loss=['mse'], metrics=['mae'])
    # model.compile(optimizer='SGD', loss=['mse', 'categorical_crossentropy'], metrics=['mae', 'accuracy']) -> pour Théo uniquement, si assez de temps disponible

    model.summary()
    return model

#### Preprocessing

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255)
dataset_white = train_datagen.flow_from_directory(
    '../../Datasets_FairFace/train',
    classes=['White'],
    target_size=(224, 224), 
    batch_size=32, 
    class_mode='input')
dataset_east_asian = train_datagen.flow_from_directory(
    '../../Datasets_FairFace/train',
    classes=['East Asian'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='input')
dataset_mixed = train_datagen.flow_from_directory(
    '../../Datasets_FairFace/train',
    target_size=(224, 224), 
    batch_size=32, 
    class_mode='binary')

Found 12287 images belonging to 1 classes.
Found 12278 images belonging to 1 classes.
Found 24565 images belonging to 2 classes.


#### Création modèle

In [14]:
# 1°) on définit le réseau en donnant un nom à la couche cachée dont on veut récupérer l'espace latent, par exemple : 
shape = (224,224,1)
input_latent = 64 #nombre de dimensions de l'espace latent, celui dont on veut qu'il modélise le face space de Tim Valentine donc
model = create_autoencoder(input_latent, shape)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 batch_normalization (BatchN  (None, 224, 224, 1)      4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 8)       968       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 8)      0         
 )                                                               
                                                                 
 batch_normalization_1 (Batc  (None, 112, 112, 8)      32        
 hNormalization)                                                 
                                                             

#### Modèle qui récupère la reconstruction de l'image

In [15]:
# 2°) On définit un deuxième modèle dontla sortie s'arrête à la layer dont on veut récupérer la représentation, on fait ça par un : 
get_latent = keras.Model(inputs=model.input, outputs=model.get_layer('latent_space_norm').output)

#### Entraînement du modèle

In [ ]:
# 3°) On entraîne le modèle : 
# cas 1, avec un autoencodeur : 
history = model.fit(x=dataset_white, y=dataset_white, validation_data=(dataset_white,dataset_white), epochs=30, batch_size=16) #adapter la taille de la batch_size à ce que peut accepter votre GPU
# cas 2, avec un hybride : 
#history = model.fit(x_train, [x_train, y_train], validation_data=(x_test, [x_test, y_test]), epochs=30, batch_size=16)

In [ ]:
# 4°) On obtient des données que l'on met en entrée du réseau par la méthode predict:
for i in range(30):
    batch = dataset_east_asian.next()
    images_batch = batch[0]
    
latent_representations = get_latent.predict(images_batch)

In [ ]:
# on le plot par PCA car le nombre de dimensions de cet espace est trop grand pour être affiché, à moins d'avoir choisi input_latent = 2 plus haut
pca = PCA(n_components=2)
latent_pca = pca.fit_transform(latent_representations)
plt.figure(figsize=(8, 6))
scatter = plt.scatter(latent_pca[:, 0], latent_pca[:, 1], c=np.argmax(y_test, axis=1), cmap='viridis', alpha=0.5)
plt.colorbar(scatter, ticks=range(10))
plt.title('PCA Projection of the Latent Space')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

# Note n°2 : cela présuppose d'avoir pû importer toute les données en même temps dans votre RAM, ce qui sera impossible avec le dataset VGG, donc il vaut mieux prendre fairface. Si nécessaire, pensez à importer les données avec un seul canal de couleur (donc en noir et blanc) et en (224,224) voire moins (par exemple 160,160). 